FNN

In [ ]:
import numpy as np
import torch
import tensorflow as tf  # Used here only to load the FashionMNIST dataset
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from fpdf import FPDF
import pickle


pdf = FPDF()
pdf.add_page()
pdf.set_font(family="Times",style="b", size=10)


# --- Load and Preprocess FashionMNIST ---
# def load_fashion_mnist():
#     (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

#     # Normalize images to the range [0, 1]
#     train_images = train_images / 255.0
#     test_images = test_images / 255.0

#     # Flatten images from (28, 28) to (784,) for the fully connected network
#     train_images = train_images.reshape(-1, 784)
#     test_images = test_images.reshape(-1, 784)

#     # Convert labels to one-hot encoding
#     train_labels = np.eye(10)[train_labels]
#     test_labels = np.eye(10)[test_labels]

#     return (train_images, train_labels), (test_images, test_labels)# Function to convert labels to one-hot encoding
def one_hot_encode(labels, num_classes=10):
    return np.eye(num_classes)[labels]

# # Function to load and preprocess FashionMNIST with a validation split
# def load_fashion_mnist(batch_size=64, val_split=0.1):
#     # Define transformations: Convert to tensor and normalize images to [0, 1]
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,), (0.5,))  # Normalizing to [-1, 1] range
#     ])
    
#     # Load datasets
#     full_train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
#     # test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
#     with open('b1.pkl', 'rb') as b1:
#         test_dataset = pickle.load(b1)
    
#     # Split train dataset into train and validation
#     val_size = int(len(full_train_dataset) * val_split)
#     train_size = len(full_train_dataset) - val_size
#     train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
    
#     # Load data into DataLoader for batching
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
#     # Convert to numpy arrays for custom training loop
#     train_images = train_dataset.dataset.data[train_dataset.indices].numpy().reshape(-1, 784) / 255.0
#     val_images = val_dataset.dataset.data[val_dataset.indices].numpy().reshape(-1, 784) / 255.0
#     test_images = test_dataset.data.numpy().reshape(-1, 784) / 255.0
    
#     # One-hot encode labels
#     train_labels = one_hot_encode(train_dataset.dataset.targets[train_dataset.indices].numpy())
#     val_labels = one_hot_encode(val_dataset.dataset.targets[val_dataset.indices].numpy())
#     test_labels = one_hot_encode(test_dataset.targets.numpy())
    
#     return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels), train_loader, val_loader, test_loader


def load_fashion_mnist(batch_size=64, val_split=0.1):
    # Define transformations: Convert to tensor and normalize images to [0, 1]
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Normalizing to [-1, 1] range
    ])
    
    # Load datasets
    full_train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
    with open('b1.pkl', 'rb') as b1:
        test_dataset = pickle.load(b1)  # Assuming test_dataset is a TensorDataset

    # Split train dataset into train and validation
    val_size = int(len(full_train_dataset) * val_split)
    train_size = len(full_train_dataset) - val_size
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    # Load data into DataLoader for batching
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Convert to numpy arrays for custom training loop
    train_images = train_dataset.dataset.data[train_dataset.indices].numpy().reshape(-1, 784) / 255.0
    val_images = val_dataset.dataset.data[val_dataset.indices].numpy().reshape(-1, 784) / 255.0
    
    # For test_dataset, assume the first element of each item is the image and the second is the label
    test_images = test_dataset.tensors[0].numpy().reshape(-1, 784) / 255.0
    test_labels = one_hot_encode(test_dataset.tensors[1].numpy())
    
    # One-hot encode labels for training and validation sets
    train_labels = one_hot_encode(train_dataset.dataset.targets[train_dataset.indices].numpy())
    val_labels = one_hot_encode(val_dataset.dataset.targets[val_dataset.indices].numpy())
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels), train_loader, val_loader, test_loader


# --- Helper functions ---
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

# --- Dense Layer ---
class Dense:
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = np.random.randn(input_dim, output_dim) * 0.01
        self.biases = np.zeros((1, output_dim))

    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.biases

    def backward(self, grad_output, learning_rate=None):
        if learning_rate:
            grad_input = np.dot(grad_output, self.weights.T)
            grad_weights = np.dot(self.input.T, grad_output)
            grad_biases = np.sum(grad_output, axis=0, keepdims=True)

            self.weights -= learning_rate * grad_weights
            self.biases -= learning_rate * grad_biases
        return grad_input

# --- Batch Normalization ---
class BatchNormalization:
    def __init__(self, num_features, epsilon=1e-5, momentum=0.9):
        self.epsilon = epsilon
        self.momentum = momentum
        self.gamma = np.ones((1, num_features))
        self.beta = np.zeros((1, num_features))

    def forward(self, x):
        self.mean = np.mean(x, axis=0)
        self.variance = np.var(x, axis=0)
        self.x_normalized = (x - self.mean) / np.sqrt(self.variance + self.epsilon)
        return self.gamma * self.x_normalized + self.beta

    def backward(self, grad_output, learning_rate):
        grad_gamma = np.sum(grad_output * self.x_normalized, axis=0, keepdims=True)
        grad_beta = np.sum(grad_output, axis=0, keepdims=True)

        self.gamma -= learning_rate * grad_gamma
        self.beta -= learning_rate * grad_beta

        return grad_output

# --- Activation Layer (ReLU) ---
class ReLU:
    def forward(self, x):
        self.input = x
        return relu(x)

    def backward(self, grad_output):
        return grad_output * relu_derivative(self.input)

# --- Dropout Layer ---
class Dropout:
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate

    def forward(self, x, training=True):
        if training:
            self.mask = (np.random.rand(*x.shape) > self.dropout_rate).astype(float)
            return x * self.mask
        return x

    def backward(self, grad_output):
        return grad_output * self.mask

class Softmax:
    def forward(self, inputs):
        # Apply softmax function
        exps = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)
        return self.output

    def backward(self, dvalues):
        # Compute gradient on softmax output
        self.dinputs = dvalues  # Gradient is passed from the loss gradient
        return self.dinputs


# --- Adam Optimizer ---
class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {}
        self.v = {}
        self.t = 0

    def update(self, layer, grad_w, grad_b):
        self.t += 1

        if layer not in self.m:
            self.m[layer] = {"w": np.zeros_like(grad_w), "b": np.zeros_like(grad_b)}
            self.v[layer] = {"w": np.zeros_like(grad_w), "b": np.zeros_like(grad_b)}

        # Update biased first moment estimate
        self.m[layer]["w"] = self.beta1 * self.m[layer]["w"] + (1 - self.beta1) * grad_w
        self.m[layer]["b"] = self.beta1 * self.m[layer]["b"] + (1 - self.beta1) * grad_b

        # Update biased second raw moment estimate
        self.v[layer]["w"] = self.beta2 * self.v[layer]["w"] + (1 - self.beta2) * (grad_w ** 2)
        self.v[layer]["b"] = self.beta2 * self.v[layer]["b"] + (1 - self.beta2) * (grad_b ** 2)

        # Correct bias in first moment
        m_w_hat = self.m[layer]["w"] / (1 - self.beta1 ** self.t)
        m_b_hat = self.m[layer]["b"] / (1 - self.beta1 ** self.t)

        # Correct bias in second moment
        v_w_hat = self.v[layer]["w"] / (1 - self.beta2 ** self.t)
        v_b_hat = self.v[layer]["b"] / (1 - self.beta2 ** self.t)

        # Update weights and biases
        layer.weights -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
        layer.biases -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)




# Helper function for accuracy
def compute_accuracy(y_pred, y_true):
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)
    return accuracy_score(y_true_classes, y_pred_classes)

# Helper function for macro-F1 score
def compute_macro_f1(y_pred, y_true):
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)
    return f1_score(y_true_classes, y_pred_classes, average='macro')

# Helper function for computing cross-entropy loss
def compute_loss(y_pred, y_true):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-12), axis=1))



# --- Neural Network ---
class NeuralNetwork:
    def __init__(self, layers, learning_rate=0.001):
        self.layers = layers
        self.learning_rate = learning_rate
        self.optimizer = AdamOptimizer(learning_rate=learning_rate)

    def forward(self, x, training=True):
        for layer in self.layers:
            x = layer.forward(x) if not isinstance(layer, Dropout) else layer.forward(x, training)
        return x

    def backward(self, grad_output):
        for layer in reversed(self.layers):
            # If layer is Dense, pass learning rate to the backward method
            if isinstance(layer, (Dense, BatchNormalization)):
                grad_output = layer.backward(grad_output, self.learning_rate)
            else:
                grad_output = layer.backward(grad_output)


    def train(self, x_train, y_train, x_val, y_val, epochs, batch_size):
        train_losses, val_losses = [], []
        train_accuracies, val_accuracies = [], []
        val_macro_f1_scores = []

        

        training_logs = []
        
        for epoch in range(epochs):
            # Mini-batch training
            for i in range(0, len(x_train), batch_size):
                x_batch, y_batch = x_train[i:i + batch_size], y_train[i:i + batch_size]
                output = self.forward(x_batch, training=True)
                loss_grad = output - y_batch
                self.backward(loss_grad)
            
            # Compute metrics for training set
            train_pred = self.predict(x_train)
            train_loss = compute_loss(train_pred, y_train)
            train_acc = compute_accuracy(train_pred, y_train)
            
            # Compute metrics for validation set
            val_pred = self.predict(x_val)
            val_loss = compute_loss(val_pred, y_val)
            val_acc = compute_accuracy(val_pred, y_val)
            val_macro_f1 = compute_macro_f1(val_pred, y_val)

            # Append metrics
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)
            val_macro_f1_scores.append(val_macro_f1)


            # # Log training details to a file
            # log_file = open("training_log.txt", "a")

            # # Inside your training loop
            # log_file.write(f"Epoch {epoch+1}/{epochs} - "
            #             f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            #             f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
            #             f"Val Macro-F1: {val_macro_f1:.4f}\n")
            
            training_logs.append(
                f"Epoch {epoch+1}/{epochs} - "
                f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                f"Val Macro-F1: {val_macro_f1:.4f}\n"
            )
            

            

            
            # Print epoch summary
            print(f"Epoch {epoch+1}/{epochs} - "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                  f"Val Macro-F1: {val_macro_f1:.4f}")
            
            # log_file.close()

        # Save training logs to a PDF file
        for log in training_logs:
            pdf.cell(0, 10, txt=log, ln=True)


        return train_losses, val_losses, train_accuracies, val_accuracies, val_macro_f1_scores

    def predict(self, x):
        return self.forward(x, training=False)
    

def plot_metrics(metrics, label, title, ylabel, filename):
    plt.figure(figsize=(14, 5))
    for metric_data in metrics:
        plt.plot(metric_data['epochs'], metric_data['values'], label=f"LR: {metric_data['learning_rate']} - Arch:")
    # plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(f"{filename}.png")  # Save plot as a PNG file for LaTeX
    plt.show()

# --- Load and Preprocess FashionMNIST ---
# --- Setup for FashionMNIST Training ---
(train_images, train_labels), (val_images, val_labels), (test_images, test_labels), train_loader, val_loader, test_loader = load_fashion_mnist()

# --- Example Setup for a Network ---
# nn = NeuralNetwork([
#     Dense(input_dim=784, output_dim=128),
#     BatchNormalization(num_features=128),
#     ReLU(),
#     Dropout(0.5),
#     Dense(input_dim=128, output_dim=64),
#     BatchNormalization(num_features=64),
#     ReLU(),
#     Dropout(0.5),
#     Dense(input_dim=64, output_dim=10),
#     Softmax()
# ])

learning_rates = [0.005, 0.001, 0.0005, 0.0001]
architectures = [
    [Dense(784, 128), BatchNormalization(128), ReLU(), Dense(128, 64), BatchNormalization(64), ReLU(), Dense(64, 10), Softmax()],
    [Dense(784, 256), BatchNormalization(256), ReLU(), Dense(256, 128), BatchNormalization(128), ReLU(), Dense(128, 10), Softmax()],
    [Dense(784, 512), BatchNormalization(512), ReLU(), Dense(512, 256), BatchNormalization(256), ReLU(), Dense(256, 10), Softmax()]
]

best_model = None
best_macro_f1 = 0
best_arch = None    
best_lr = None

results = []
for lr in learning_rates:
    for arch in architectures:
        nn = NeuralNetwork(arch, learning_rate=lr)
        
        # Train the network
        train_losses, val_losses, train_accuracies, val_accuracies, val_macro_f1_scores = nn.train(
            train_images, train_labels, val_images, val_labels, epochs=10, batch_size=64
        )

        # Collect metrics
        results.append({
            'learning_rate': lr,
            'architecture': arch,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracies': train_accuracies,
            'val_accuracies': val_accuracies,
            'val_macro_f1_scores': val_macro_f1_scores,
            'nn_model': nn  # Save the model for generating confusion matrix later
        })
        
        # Track the best model based on validation macro-F1 score
        max_f1 = max(val_macro_f1_scores)
        if max_f1 > best_macro_f1:
            best_macro_f1 = max_f1
            best_model = nn
            best_arch = arch
            best_lr = lr

            
            
        # Plot training curves
        plt.figure(figsize=(14, 5))
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label="Train Loss")
        plt.plot(val_losses, label="Validation Loss")
        plt.title(f"Learning Rate {lr} - Loss")
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(train_accuracies, label="Train Accuracy")
        plt.plot(val_accuracies, label="Validation Accuracy")
        plt.title(f"Learning Rate {lr} - Accuracy")
        plt.legend()
        plt.show()


plot_count = 0

# Prepare data for plotting
for result in results:
    lr = result['learning_rate']
    arch = result['architecture']
    epochs = range(1, len(result['train_losses']) + 1)
    
    # Plot Loss
    plot_metrics([{'epochs': epochs, 'values': result['train_losses'], 'learning_rate': lr, 'arch': str(arch)}], 
                 'Training Loss', f"Loss vs Epoch (LR: {lr}, Arch: {arch})", "Loss", f"train_loss_plot_{plot_count}")
    
    
    # Plot Accuracy
    plot_metrics([{'epochs': epochs, 'values': result['train_accuracies'], 'learning_rate': lr, 'arch': str(arch)}], 
                 'Training Accuracy', f"Accuracy vs Epoch (LR: {lr}, Arch: {arch})", "Accuracy", f"train_accuracy_plot_{plot_count}")
    
    # Plot F1 Score
    plot_metrics([{'epochs': epochs, 'values': result['val_macro_f1_scores'], 'learning_rate': lr, 'arch': str(arch)}], 
                 'Validation Macro F1 Score', f"F1 Score vs Epoch (LR: {lr}, Arch: {arch})", "F1 Score", f"val_f1_score_plot_{plot_count}")



confusion_plot_count = 0

for result in results:
    nn = result['nn_model']
    lr = result['learning_rate']
    arch = result['architecture']
    
    # Generate predictions and confusion matrix for the validation set
    val_predictions = nn.predict(val_images)
    val_predictions = np.argmax(val_predictions, axis=1)  # Get the predicted class
    y_true = np.argmax(val_labels, axis=1)  # Assuming one-hot encoded labels

    cm = confusion_matrix(y_true, val_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
    plt.title(f"Confusion Matrix (LR: {lr}, Arch)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.savefig(f"confusion_matrix_{confusion_plot_count}.png")
    plt.show()

    pdf.add_page()
    pdf.image(f"confusion_matrix_{confusion_plot_count}.png", x=10, y=10, w=180)
    confusion_plot_count += 1


# Get predictions from the best model on the validation set
val_pred = best_model.predict(val_images)
val_pred_classes = np.argmax(val_pred, axis=1)
val_true_classes = np.argmax(val_labels, axis=1)

# Plot confusion matrix
cm = confusion_matrix(val_true_classes, val_pred_classes)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Best Model on Validation Set")
plt.savefig("confusion_matrix_best_model.png")
plt.show()

pdf.add_page()
pdf.image("confusion_matrix_best_model.png", x=10, y=10, w=180)




# Evaluate the best model on the test set
test_pred = best_model.predict(test_images)
test_loss = compute_loss(test_pred, test_labels)
test_acc = compute_accuracy(test_pred, test_labels)
test_macro_f1 = compute_macro_f1(test_pred, test_labels)

test_results = f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, Test Macro-F1: {test_macro_f1:.4f}"

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, Test Macro-F1: {test_macro_f1:.4f}")

# Add a separator line before test results
pdf.add_page()
pdf.cell(0, 10, "", ln=True)  # Blank line
pdf.cell(0, 10, "Test Results:", ln=True)  # Heading for the test results
pdf.cell(0, 10, test_results, ln=True)  # Add test results to the PDF

pdf.output("training_report.pdf")  # Save the PDF file
print("Training details, test results, and confusion matrices saved to training_report.pdf.")





/tmp/ipykernel_16879/3879243728.py:363: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt=log, ln=True)
/tmp/ipykernel_16879/3879243728.py:363: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt=log, ln=True)


Test Loss: 2.1372, Test Accuracy: 0.4661, Test Macro-F1: 0.2523


/tmp/ipykernel_16879/3879243728.py:363: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt=log, ln=True)
/tmp/ipykernel_16879/3879243728.py:363: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt=log, ln=True)


Test Loss: 1.9973, Test Accuracy: 0.4993, Test Macro-F1: 0.2760


KeyboardInterrupt: 

Pickle

In [14]:
import pickle

def clear_intermediate_outputs(model):
    """
    Clears the intermediate input and output data from each layer to save space.
    """
    for layer in model.layers:
        # Check if the layer has attributes for intermediate values, and clear them
        if hasattr(layer, 'input'):
            layer.input = None
        if hasattr(layer, 'output'):
            layer.output = None
        # If there are additional large attributes, clear them similarly:
        if hasattr(layer, 'x_normalized'):  # For BatchNormalization layers
            layer.x_normalized = None
        if hasattr(layer, 'mean'):
            layer.mean = None
        if hasattr(layer, 'variance'):
            layer.variance = None

def predict_with_loaded_model(model, query_images):
    predictions = model.predict(query_images)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes


# Clear intermediate data from the model
clear_intermediate_outputs(best_model)

# Save the best model to disk
with open("best_model_nn.pkl", "wb") as file:
    pickle.dump(best_model, file)

# Load the best model from disk
with open("best_model_nn.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Predict with the loaded model
query_images = test_images
predicted_classes = predict_with_loaded_model(loaded_model, query_images)

print(predicted_classes)


[9 2 1 ... 8 1 5]


In [4]:
import pickle

def predict_with_loaded_model(model, query_images):
    predictions = model.predict(query_images)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes

# Save the best model to disk
with open("best_model.pkl", "wb") as file:
    pickle.dump(best_model, file)

# Load the best model from disk
with open("best_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Predict with the loaded model
query_images = test_images
predicted_classes = predict_with_loaded_model(loaded_model, query_images)

print(predicted_classes)



[9 2 1 ... 8 1 5]


In [13]:
import pickle
import numpy as np

# Save only the model weights (gamma, beta, etc.) to a file
def save_model_weights(model, filename="best_model_weights.pkl"):
    weights = {}
    for i, layer in enumerate(model.layers):
        if hasattr(layer, 'gamma') and hasattr(layer, 'beta'):
            weights[f"layer_{i}_gamma"] = layer.gamma
            weights[f"layer_{i}_beta"] = layer.beta
        if hasattr(layer, 'weights'):
            weights[f"layer_{i}_weights"] = layer.weights
            weights[f"layer_{i}_biases"] = layer.biases
    with open(filename, "wb") as file:
        pickle.dump(weights, file)

save_model_weights(best_model)



# Load weights into a new model instance
def load_model_weights(model, filename="best_model_weights.pkl"):
    with open(filename, "rb") as file:
        weights = pickle.load(file)
    
    for i, layer in enumerate(model.layers):
        if hasattr(layer, 'gamma') and hasattr(layer, 'beta'):
            layer.gamma = weights[f"layer_{i}_gamma"]
            layer.beta = weights[f"layer_{i}_beta"]
        if hasattr(layer, 'weights'):
            layer.weights = weights[f"layer_{i}_weights"]
            layer.biases = weights[f"layer_{i}_biases"]


def predict_with_loaded_model(model, query_images):
    predictions = model.predict(query_images)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes




# Test Block (for loading and using the model)
if __name__ == "__main__":
    # Load the trained model weights
    nn = NeuralNetwork(best_arch, best_lr)  # Ensure to create the same architecture
    load_model_weights(nn, 'best_model_weights.pkl')  # Load the saved weights

    # Assuming you have test images to classify
    query_images = test_images  # Replace with actual query images

    # Predict labels for the query images
    predicted_labels = predict_with_loaded_model(nn, query_images)

    # Print or return predicted labels
    print(predicted_labels)



[9 2 1 ... 8 1 5]
